In [2]:
import pandas as pd
from Population import Individual,Population
import plotly.express as px
import plotly.graph_objects as go
import os

In [1]:
# Evolutionary algorythm

population = Population(size=200)
for generation in range(80):
    population.next_generation(mutation_rate=0.1)

NameError: name 'Population' is not defined

In [ ]:

# Simulate genes
genes = {'Munchen': 400, 'Munchen_battery': 100, 'Bulgaria': 0, 'Bulgaria_battery': 0, 'Bulgaria-Munchen': 0, 'Italy': 0, 'Italy_battery': 0, \
         'Italy-Munchen': 0, 'Spain': 0, 'Spain_battery': 0, 'Spain-Munchen': 0, 'VlakteRaan': 0, 'VlakteRaan_battery': 0, 'VlakteRaan-Munchen': 0, 
         'Arkona': 0, 'Arkona_battery': 0, 'Arkona-Munchen': 0}
top_gene = Individual(genes=list(genes.values()))
df = top_gene.run_simulation(get_df=True)

# Create a DataFrame with the datetime index
plotter = pd.DataFrame({'Value': df.Balance.values})

# Create an interactive line plot using Plotly Express
fig = px.line(plotter, x=plotter.index, y='Value', title=f'Cost: {top_gene.base_cost}')

# Show the interactive graph
fig.show(renderer='browser')

In [28]:
# Create subfolder for PNG files
figs_to_save = []

# Data definitions
coords = {
    'Munchen': (48.1, 11.6, 'Solar'),
    'Munchen_battery': (48.1, 10.6, 'Battery'),
    'Bulgaria': (42, 26, 'Solar'),
    'Bulgaria_battery': (42, 25, 'Battery'),
    'Italy': (37.5, 14.9, 'Solar'),
    'Italy_battery': (37.5, 13.9, 'Battery'),
    'Spain': (38.2, -5, 'Solar'),
    'Spain_battery': (38.2, -6, 'Battery'),
    'Arkona': (54.8, 13.5, 'Wind'),
    'Arkona_battery': (54.8, 12.5, 'Battery'),
    'VlakteRaan': (51.6, 4, 'Wind'),
    'VlakteRaan_battery': (51.6, 3, 'Battery')
}

color_map = {
    'Solar': 'gold',
    'Wind': 'royalblue',    
    'Battery': 'forestgreen' 
}

df_coords = pd.DataFrame.from_dict(coords, orient='index', columns=["lat", "lon", "Type"])

# Visualize evolution
top_sim_df = pd.read_csv('optimal_evolution.csv')
for idx, row in top_sim_df.iterrows():
    # Filter generation values and join with coordinates
    generation_values = row.iloc[2:][~row.iloc[2:].index.str.contains('-')].to_frame(name="GW")
    df_merged = generation_values.join(df_coords, how="left")
    order = {'Battery': 0, 'Wind': 1, 'Solar': 2}
    df_merged = df_merged.sort_values(by='Type', key=lambda col: col.map(order))
    
    # Create a mapbox-based scatter plot
    scatter_fig = px.scatter_map(
        df_merged, 
        lat="lat", 
        lon="lon",
        size="GW", 
        color='Type',
        color_discrete_map=color_map,
        zoom=4, 
        map_style="open-street-map",
        size_max=50, 
        title=f'Generation: {row["Generation"]}, Cost: {row["Total Cost"]}',
        opacity=1
    )

    
    # Add an additional mapbox trace (instead of a geo trace)
    scatter_fig.add_trace(
        go.Scattermap(
            lon=[-5, 10.6],
            lat=[38.2, 48.1],
            mode='lines',
            line=dict(width=row['Spain-Munchen']/100, color='black'),
            name="Spain",
            opacity=0.7,
            showlegend=False
        )
    )

    scatter_fig.add_trace(
        go.Scattermap(
            lon=[14.9, 10.6],
            lat=[37.5, 48.1],
            mode='lines',
            line=dict(width=row['Italy-Munchen']/100, color='black'),
            name="Italy",
            opacity=0.7,
            showlegend=False
        )
    )

    scatter_fig.add_trace(
        go.Scattermap(
            lon=[26, 10.6],
            lat=[42, 48.1],
            mode='lines',
            line=dict(width=row['Bulgaria-Munchen']/100, color='black'),
            name="Bulgaria",
            opacity=0.7,
            showlegend=False
        )
    )

    scatter_fig.add_trace(
        go.Scattermap(
            lon=[13.5, 10.6],
            lat=[54.8, 48.1],
            mode='lines',
            line=dict(width=row['Arkona-Munchen']/100, color='black'),
            name="Arkona",
            opacity=0.7,
            showlegend=False
        )
    )

    scatter_fig.add_trace(
        go.Scattermap(
            lon=[4, 10.6],
            lat=[51.6, 48.1],
            mode='lines',
            line=dict(width=row['VlakteRaan-Munchen']/100, color='black'),
            name="Vlakte van Raan",
            opacity=0.7,
            showlegend=False
        )
    )

    figs_to_save.append(scatter_fig)


In [63]:
figs_to_save[-1].show(renderer='browser')

In [70]:
png_folder = r'C:\Users\matde\Downloads\gif_evolution'
png_files = sorted([os.path.join(png_folder, f) for f in os.listdir(png_folder) if f.endswith(".png")])
frames = [imageio.imread(png) for png in png_files]
imageio.mimsave("map_animation.gif", frames, duration=0.2, loop=1)

C:\Users\matde\AppData\Local\Temp\ipykernel_20948\3663568946.py:3: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.



In [74]:
from PIL import Image
import os

png_files = sorted([os.path.join(png_folder, f) for f in os.listdir(png_folder) if f.endswith(".png")])
frames = [Image.open(png) for png in png_files]

# Append an extra copy of the final frame.
frames.append(frames[-1])

# Set durations: 200ms for each normal frame, and 2000ms for the extra (final) frame.
durations = [200] * (len(frames) - 1) + [2000]

frames[0].save("map_animation.gif",
               format="GIF",
               append_images=frames[1:],
               save_all=True,
               duration=durations,
               loop=0)
